<a href="https://colab.research.google.com/github/derekwin/colab/blob/main/snn/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/snntorch_alpha_w.png?raw=true' width="300">](https://github.com/jeshraghian/snntorch/)

# Quickstart with snnTorch
### By Jason K. Eshraghian (www.jasoneshraghian.com)

<a href="https://colab.research.google.com/github/jeshraghian/snntorch/blob/master/examples/quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

[<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/GitHub-Mark-Light-120px-plus.png?raw=true' width="28">](https://github.com/jeshraghian/snntorch/) [<img src='https://github.com/jeshraghian/snntorch/blob/master/docs/_static/img/GitHub_Logo_White.png?raw=true' width="80">](https://github.com/jeshraghian/snntorch/)

For a comprehensive overview on how SNNs work, and what is going on under the hood, [then you might be interested in the snnTorch tutorial series available here.](https://snntorch.readthedocs.io/en/latest/tutorials/index.html)
The snnTorch tutorial series is based on the following paper. If you find these resources or code useful in your work, please consider citing the following source:

> <cite> [Jason K. Eshraghian, Max Ward, Emre Neftci, Xinxin Wang, Gregor Lenz, Girish Dwivedi, Mohammed Bennamoun, Doo Seok Jeong, and Wei D. Lu. "Training Spiking Neural Networks Using Lessons From Deep Learning". Proceedings of the IEEE, 111(9) September 2023.](https://ieeexplore.ieee.org/abstract/document/10242251) </cite>

In [14]:
!pip install snntorch --quiet

In [ ]:
import torch, torch.nn as nn
import snntorch as snn

## DataLoading
Define variables for dataloading.

In [ ]:
batch_size = 128
data_path='/tmp/data/mnist'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Load MNIST dataset.

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define a transform
transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True)

## Define Network with snnTorch.
* `snn.Leaky()` instantiates a simple leaky integrate-and-fire (LIF_ neuron.
* `spike_grad` optionally defines the surrogate gradient. Defaults to the arctangent surrogate gradient.

By default, each LIF neuron returns two values: the spike and hidden state.
But neurons chained together in `nn.Sequential` expect only one value. To handle this:

* `init_hidden` initializes the hidden states (e.g., membrane potential) as instance variables to be processed in the background.

The final layer is not bound by this constraint, and can return multiple tensors:
* `output=True` enables the final layer to return the hidden state in addition to the spike.

In [ ]:
from snntorch import surrogate

beta = 0.9  # neuron decay rate
spike_grad = surrogate.fast_sigmoid() # fast sigmoid surrogate gradient

#  Initialize Convolutional SNN
net = nn.Sequential(nn.Conv2d(1, 8, 5),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Conv2d(8, 16, 5),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Flatten(),
                    nn.Linear(16*4*4, 10),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True)
                    ).to(device)

Refer to the snnTorch documentation to see more [neuron types](https://snntorch.readthedocs.io/en/latest/snntorch.html) and [surrogate gradient options](https://snntorch.readthedocs.io/en/latest/snntorch.surrogate.html).

## Define the Forward Pass
Now define the forward pass over multiple time steps of simulation.

In [ ]:
from snntorch import utils

def forward_pass(net, data, num_steps):
  spk_rec = [] # record spikes over time
  utils.reset(net)  # reset/initialize hidden states for all LIF neurons in net

  for step in range(num_steps): # loop over time
      spk_out, mem_out = net(data) # one time step of the forward-pass
      spk_rec.append(spk_out) # record spikes

  return torch.stack(spk_rec)

Define the optimizer and loss function. Here, we use the MSE Count Loss, which counts up the total number of output spikes at the end of the simulation run. The correct class has a target firing rate of 80% of all time steps, and incorrect classes are set to 20%.

In [ ]:
import snntorch.functional as SF

optimizer = torch.optim.Adam(net.parameters(), lr=2e-3, betas=(0.9, 0.999))
loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)

Objective functions do not have to be applied to the spike count. They may be applied to the membrane potential (hidden state), or to spike-timing targets instead of rate-based methods. A non-exhaustive list of objective functions available include:

**Apply the objective directly to spikes:**
* MSE Spike Count Loss: `mse_count_loss()`
* Cross Entropy Spike Count Loss: `ce_count_loss()`
* Cross Entropy Spike Rate Loss: `ce_rate_loss()`

**Apply the objective to the hidden state:**
* Cross Entropy Maximum Membrane Potential Loss: `ce_max_membrane_loss()`
* MSE Membrane Potential Loss: `mse_membrane_loss()`

For alternative objective functions, refer to the `snntorch.functional` [documentation here.](https://snntorch.readthedocs.io/en/latest/snntorch.functional.html)

## Training Loop

Now for the training loop. The predicted class will be set to the neuron with the highest firing rate, i.e., a rate-coded output. We will just measure accuracy on the training set. This training loop follows the same syntax as with PyTorch.

In [ ]:
num_epochs = 1 # run for 1 epoch - each data sample is seen only once
num_steps = 25  # run for 25 time steps

loss_hist = [] # record loss over iterations
acc_hist = [] # record accuracy over iterations

# training loop
for epoch in range(num_epochs):
    for i, (data, targets) in enumerate(iter(train_loader)):
        data = data.to(device)
        targets = targets.to(device)

        net.train()
        spk_rec = forward_pass(net, data, num_steps) # forward-pass
        loss_val = loss_fn(spk_rec, targets) # loss calculation
        optimizer.zero_grad() # null gradients
        loss_val.backward() # calculate gradients
        optimizer.step() # update weights
        loss_hist.append(loss_val.item()) # store loss

        # print every 25 iterations
        if i % 25 == 0:
          print(f"Epoch {epoch}, Iteration {i} \nTrain Loss: {loss_val.item():.2f}")

          # check accuracy on a single batch
          acc = SF.accuracy_rate(spk_rec, targets)
          acc_hist.append(acc)
          print(f"Accuracy: {acc * 100:.2f}%\n")

        # uncomment for faster termination
        # if i == 150:
        #     break


## More control over your model
If you are simulating more complex architectures, such as residual nets, then your best bet is to wrap the network up in a class as shown below. This time, we will explicitly use the membrane potential, `mem`, and let `init_hidden` default to false.

For the sake of speed, we'll just simulate a fully-connected SNN, but this can be generalized to other network types (e.g., Convs).

In addition, let's set the neuron decay rate, `beta`, to be a learnable parameter. The first layer will have a shared decay rate across neurons. Each neuron in the second layer will have an independent decay rate. The decay is clipped between [0,1].

In [27]:
import torch.nn.functional as F

# Define Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        num_inputs = 784 # number of inputs
        num_hidden = 300 # number of hidden neurons
        num_hidden2 = 100 # number of hidden neurons
        num_outputs = 10 # number of classes (i.e., output neurons)

        beta1 = 0.9 # global decay rate for all leaky neurons in layer 1
        beta2 = torch.rand((num_hidden2), dtype = torch.float) # independent decay rate for each leaky neuron in layer 2: [0, 1)
        beta3 = torch.rand((num_outputs), dtype = torch.float)

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta1) # not a learnable decay rate
        self.fc2 = nn.Linear(num_hidden, num_hidden2)
        self.lif2 = snn.Leaky(beta=beta2, learn_beta=True) # learnable decay rate
        self.fc3 = nn.Linear(num_hidden2, num_outputs)
        self.lif3 = snn.Leaky(beta=beta3, learn_beta=True) # learnable decay rate

    def forward(self, x):
        mem1 = self.lif1.init_leaky() # reset/init hidden states at t=0
        mem2 = self.lif2.init_leaky() # reset/init hidden states at t=0
        mem3 = self.lif3.init_leaky() # reset/init hidden states at t=0
        spk2_rec = [] # record output spikes
        mem2_rec = [] # record output hidden states

        for step in range(num_steps): # loop over time
            cur1 = self.fc1(x.flatten(1))
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)

            spk2_rec.append(spk3) # record spikes
            mem2_rec.append(mem3) # record membrane

        return torch.stack(spk2_rec), torch.stack(mem2_rec)

# Load the network onto CUDA if available
net = Net().to(device)

In [28]:
optimizer = torch.optim.Adam(net.parameters(), lr=2e-3, betas=(0.9, 0.999))
loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)

num_epochs = 1 # run for 1 epoch - each data sample is seen only once
num_steps = 25  # run for 25 time steps

loss_hist = [] # record loss over iterations
acc_hist = [] # record accuracy over iterations

# training loop
for epoch in range(num_epochs):
    for i, (data, targets) in enumerate(iter(train_loader)):
        data = data.to(device)
        targets = targets.to(device)

        net.train()
        spk_rec, _ = net(data) # forward-pass
        loss_val = loss_fn(spk_rec, targets) # loss calculation
        optimizer.zero_grad() # null gradients
        loss_val.backward() # calculate gradients
        optimizer.step() # update weights
        loss_hist.append(loss_val.item()) # store loss

        # print every 25 iterations
        if i % 25 == 0:
          net.eval()
          print(f"Epoch {epoch}, Iteration {i} \nTrain Loss: {loss_val.item():.2f}")

          # check accuracy on a single batch
          acc = SF.accuracy_rate(spk_rec, targets)
          acc_hist.append(acc)
          print(f"Accuracy: {acc * 100:.2f}%\n")

        # uncomment for faster termination
        # if i == 150:
        #     break


Epoch 0, Iteration 0 
Train Loss: 2.50
Accuracy: 12.50%

Epoch 0, Iteration 25 
Train Loss: 0.98
Accuracy: 23.44%

Epoch 0, Iteration 50 
Train Loss: 0.69
Accuracy: 43.75%

Epoch 0, Iteration 75 
Train Loss: 0.45
Accuracy: 77.34%

Epoch 0, Iteration 100 
Train Loss: 0.40
Accuracy: 82.03%

Epoch 0, Iteration 125 
Train Loss: 0.35
Accuracy: 86.72%

Epoch 0, Iteration 150 
Train Loss: 0.29
Accuracy: 89.06%

Epoch 0, Iteration 175 
Train Loss: 0.29
Accuracy: 85.16%

Epoch 0, Iteration 200 
Train Loss: 0.25
Accuracy: 87.50%

Epoch 0, Iteration 225 
Train Loss: 0.19
Accuracy: 92.19%

Epoch 0, Iteration 250 
Train Loss: 0.22
Accuracy: 88.28%

Epoch 0, Iteration 275 
Train Loss: 0.15
Accuracy: 92.19%

Epoch 0, Iteration 300 
Train Loss: 0.11
Accuracy: 96.09%

Epoch 0, Iteration 325 
Train Loss: 0.11
Accuracy: 95.31%

Epoch 0, Iteration 350 
Train Loss: 0.17
Accuracy: 89.84%

Epoch 0, Iteration 375 
Train Loss: 0.12
Accuracy: 93.75%

Epoch 0, Iteration 400 
Train Loss: 0.14
Accuracy: 94.53%

Ep

In [30]:
print(f"Trained decay rate of the first layer: {net.lif1.beta:.3f}\n")

print(f"Trained decay rates of the second layer: {net.lif2.beta}\n")

print(f"Trained decay rates of the third layer: {net.lif3.beta}")

Trained decay rate of the first layer: 0.900

Trained decay rates of the second layer: Parameter containing:
tensor([ 5.7000e-01,  9.5821e-01,  1.0153e+00,  2.0020e-01,  7.9383e-01,
         4.3443e-01,  3.6618e-01,  3.7078e-02,  9.6803e-01,  8.1851e-02,
         9.7122e-01, -2.1681e-04,  7.8992e-02,  8.4287e-01,  4.4065e-01,
         6.4002e-01,  4.2385e-01,  7.4808e-01,  9.0783e-01,  1.0147e+00,
         7.2968e-01,  8.8453e-01,  4.6594e-01,  2.6593e-01,  3.6361e-01,
         2.8172e-01,  1.8668e-01,  7.2068e-01,  3.7378e-01,  3.9678e-01,
         6.0693e-01,  3.3597e-01,  8.0685e-01,  1.7363e-01,  6.6681e-01,
         1.7579e-01,  3.9498e-01,  7.9376e-02,  6.3900e-01,  5.5194e-02,
         4.0792e-01,  1.0386e-01,  5.2506e-01,  8.1080e-01,  7.0446e-01,
         6.5138e-01,  2.8092e-01,  3.4020e-01,  7.2027e-01,  6.0119e-01,
         4.2353e-01,  2.4460e-01,  2.8391e-02,  5.2256e-01,  9.0260e-01,
         8.0490e-01,  4.1735e-01,  6.6884e-01,  8.5019e-01,  9.1405e-01,
         9.7266

In [31]:
# function to measure accuracy on full test set
def test_accuracy(data_loader, net, num_steps):
  with torch.no_grad():
    total = 0
    acc = 0
    net.eval()

    data_loader = iter(data_loader)
    for data, targets in data_loader:
      data = data.to(device)
      targets = targets.to(device)
      spk_rec, _ = net(data)

      acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
      total += spk_rec.size(1)

  return acc/total

In [32]:
print(f"Test set accuracy: {test_accuracy(test_loader, net, num_steps)*100:.3f}%")

Test set accuracy: 94.940%


# Conclusion
That's it for the quick intro to snnTorch!

* For a detailed tutorial of spiking neurons, neural nets, encoding, and training using neuromorphic datasets, check out the
[snnTorch tutorial series](https://snntorch.readthedocs.io/en/latest/tutorials/index.html).
* For more information on the features of snnTorch, check out the [documentation at this link](https://snntorch.readthedocs.io/en/latest/).
* If you have ideas, suggestions or would like to find ways to get involved, then [check out the snnTorch GitHub project here.](https://github.com/jeshraghian/snntorch)